# Tuning Pipeline

In [1]:
from sklearn import set_config; set_config(display='diagram')

👇 Consider the following dataset.

In [2]:
import pandas as pd

data = pd.read_csv("data.csv")

data.head()

,games played,minutes played,points per game,field goals made,field goal attempts,field goal percent,3 point made,3 point attempt,3 point %,free throw made,free throw attempts,free throw %,offensive rebounds,defensive rebounds,rebounds,assists,steals,blocks,turnovers,target_5y
0,36.0,27.4,7.4,2.6,7.6,NaN,0.5,2.1,25.0,1.6,2.3,69.9,0.7,3.4,4.1,1.9,0.4,0.4,1.3,0
1,35.0,26.9,NaN,2.0,6.7,29.6,0.7,2.8,23.5,2.6,3.4,76.5,0.5,2.0,2.4,3.7,1.1,0.5,1.6,0
2,NaN,15.3,5.2,2.0,4.7,42.2,0.4,1.7,24.4,0.9,1.3,67.0,0.5,1.7,2.2,1.0,0.5,0.3,1.0,0
3,58.0,11.6,5.7,2.3,5.5,42.6,0.1,0.5,22.6,0.9,1.3,68.9,1.0,0.9,1.9,0.8,0.6,0.1,1.0,1
4,48.0,11.5,4.5,1.6,3.0,52.4,0.0,0.1,0.0,1.3,1.9,67.4,1.0,1.5,2.5,0.3,0.3,0.4,0.8,1


In [9]:
X_train = X[:1100]
y_train = y[:1100]
X_test = X[1100:]
y_test = y[1100:]

- Each observations represents a player
- Each column a characteristic of performance

The target defines whether the player last less than 5 years `0` or 5 years or more `1` as a professional.

In [3]:
X = data.drop(columns="target_5y")
y = data['target_5y']

## Pipeline

👇 We are giving you the simple pipeline below

In [4]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer

# Preprocessing pipe
preprocessor = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaling', MinMaxScaler())
])

# Final pipe
pipe = Pipeline([
    ('preprocessing', preprocessor),
    ('model_svm', SVC())])

pipe

Pipeline(steps=[('preprocessing',
                 Pipeline(steps=[('imputer', SimpleImputer()),
                                 ('scaling', MinMaxScaler())])),
                ('model_svm', SVC())])

## Fine Tuning

Our task is to assist the recruitment process of promising young players.  
The model should **limit false alarms as much as possible** to avoid recruiting players that will flop.

❓ **Fine-tune this pipeline so as to maximize your objective**

- Use the `scoring` metric appropriate for the task
- Grid Search for the optimal:
    - imputing `strategy`
    - `kernel`
    - regularization factor `C`... 


- Store your random search results in a `search`

In [15]:
import numpy as np
np.logspace(-3, 2, 6)

array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02])

In [17]:
from sklearn import set_config; set_config(display='text')
# pipe.get_params()
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(
    pipe, 
    param_grid={
        # Access any component of the Pipeline, as far back as you want
        'preprocessing__imputer__strategy': ['mean', 'median'],
        'model_svm__kernel': ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed'],
        'model_svm__C': np.logspace(-3, 2, 6)
    },
    cv=5,
    scoring="precision")

grid_search.fit(X_train, y_train)

grid_search.best_params_

/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
60 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/site-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.7/site-packages/sklearn/svm/_base.py", line 217, in fit
    " Input is a {}x{} matrix.".format(X.shape[0], X.shape[1])
ValueError: Precompu

{'model_svm__C': 0.01,
 'model_svm__kernel': 'poly',
 'preprocessing__imputer__strategy': 'mean'}

In [12]:
pipe_tuned = grid_search.best_estimator_
pipe_tuned.fit(X_train,y_train)

# Make predictions
pipe_tuned.predict(X_test.iloc[0:2])

# Score model
pipe_tuned.score(X_test,y_test)

0.6916666666666667

In [13]:
from sklearn.model_selection import cross_val_score

# Cross-validate Pipeline
cross_val_score(pipe_tuned, X_train, y_train, cv=5, scoring='precision').mean()

0.7539520536749584

In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult('solution',
    search = search
)
result.write()
print(result.check())


## Export

Once you have built your optimal pipeline, export it as a pickle file

In [ ]:
import pickle

# Export pipeline as pickle file
with open("pipeline.pkl", "wb") as file:
    pickle.dump(pipe_tuned, file )

🏁 Congratulation. Don't forget to add, commit and push your notebook.